<a href="https://colab.research.google.com/github/wfsilva-uea/luigi/blob/master/luigi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Oque é ETL?


A coleta dos dados corrobora para o processo de Extract, Transform and Load (ETL), que são as rotinas para extração, transformação e carga dos dados extraídos dos bancos de dados de origem (transacionais) para o “Data Warehouse” (DW). O processo de ETL é a etapa mais importante dentro de processo de BI. É essencial na modelagem da base de dados e faz a conexão entre o operacional e o DW. 

### Por que sua utilização no processo de Ciências dos Dados?


Quando trabalhamos com um grande volume de dados uma ferramenta ETL deve ser utilizada. Utiliza-se da criação de consultas em SQL, de uma forma bem visual, criando componentes que irão automatizar a atualização dos dados, tabelas e visualizações a serem enviados, populando e estruturando do DW.

### Caraterísticas gerais da ferramenta. Vantagem e desvantagem

Luigi é uma biblioteca de pipeline projetada totalmente em Python pelo Spotify para resolver todos os problemas de pipeline associados a processos em lote de longa execução.
A estrutura de um pipeline em Luigi lembra a de um grafo, com nós e arestas conectando os nós.

### Desenvolver um exemplo do processo ETL, incluir no exemplo operação de linha, classificação, junção e agregação.

In [2]:
!pip install luigi

     |████████████████████████████████| 1.2MB 6.6MB/s 
  Created wheel for luigi: filename=luigi-3.0.2-cp36-none-any.whl size=1076734 sha256=e6f07e2b1987feeda319f4620c3976878fceeeb0ec4fd9540e3b00b7f4c7b8fe
  Stored in directory: /root/.cache/pip/wheels/9f/70/d2/58c9f0bba29e7123f984ba2bb927cf8877c89c8c3c82a57ade
Successfully built luigi


In [3]:
!npx degit wfsilva-uea/luigi/northwind.sqlite northwind.sqlite -f

npx: installed 1 in 0.919s
! ENOTDIR: not a directory, scandir 'northwind.sqlite'


In [4]:
import luigi
import sqlite3
import pandas as pd
import sqlalchemy as sqla

from datetime import datetime

In [5]:
# northwind connection
conn = sqlite3.connect('northwind.sqlite')

In [6]:
# dw connection
conn_dw = sqlite3.connect('dw.sqlite')

In [7]:
class CustomerTask(luigi.Task):
  
  def output(self):
    today_str = datetime.today().strftime('%d%m%Y')
    return luigi.LocalTarget('customers_%s.log' % today_str)

  def run(self):
    with self.output().open('w') as target:

      target.write('Iniciando processo ETL\n')

      sql = """
        select 
          customer_id,
          contact_name,
          contact_title,
          company_name
        from customers
        order by contact_name  
      """

      target.write('Lendo tabela de clientes\n')

      df = pd.read_sql_query(sql, conn)
      
      target.write('%d registros foram encontrados\n' % len(df))

      target.write('Gerando tabela clientes no DW\n')
      
      # storing in CSV
      df.to_csv('clientes.csv', index=False, encoding='utf-8')

      target.write('Finalizando processo ETL\n')

In [8]:
class ProductsTask(luigi.Task):
  
  def requires(self):
    return CustomerTask()

  def output(self):
    today_str = datetime.today().strftime('%d%m%Y')
    return luigi.LocalTarget('products_%s.log' % today_str)

  def run(self):
    with self.output().open('w') as target:

      target.write('Iniciando processo ETL\n')

      sql = """
        select 
          product_id,
          product_name,
          unit_price
        from products
        order by product_name
      """

      target.write('Lendo tabela de produtos\n')

      df = pd.read_sql_query(sql, conn)
      
      target.write('%d registros foram encontrados\n' % len(df))

      target.write('Gerando tabela produtos no DW\n')
      
      # storing in DW
      df.to_sql('produtos', index=False, con=conn_dw, if_exists='replace')

      target.write('Finalizando processo ETL\n')

In [9]:
class OrderTask(luigi.Task):
  
  def requires(self):
    return ProductsTask()

  def output(self):
    today_str = datetime.today().strftime('%d%m%Y')
    return luigi.LocalTarget('orders_%s.log' % today_str)

  def run(self):
    with self.output().open('w') as target:

      target.write('Iniciando processo ETL\n')

      sql = """
        select
          c.customer_id,
          p.product_id,
          sum((od.unit_price * od.quantity * (1 - od.discount) / 100) * 100) as subtotal
        from products p 
        inner join order_details od on p.product_id = od.product_id
        inner join orders o on od.order_id = o.order_id
        inner join customers c on o.customer_id = c.customer_id
        where o.order_date between '1996-01-01' and '1996-12-31'
        group by c.customer_id, p.product_id
      """

      # orders
      target.write('Lendo tabela de pedidos\n')

      df_orders = pd.read_sql_query(sql, conn)
      
      target.write('%d registros foram encontrados\n' % len(df_orders))

      
      # customers
      target.write('Lendo arquivo CSV de clientes\n')

      df_customers = pd.read_csv('clientes.csv', encoding='utf-8')

      target.write('%d registros foram encontrados\n' % len(df_customers))


      # products
      target.write('Lendo tabela de produtos\n')

      df_products = pd.read_sql_query('select * from produtos', conn_dw)

      target.write('%d registros foram encontrados\n' % len(df_products))


      # merging orders and customers
      target.write('Realizando merge das tabela de clientes, produtos e pedidos\n')

      df_merged = pd.merge(df_orders, df_customers, left_on=['customer_id'], right_on=['customer_id'])    
      df_merged = pd.merge(df_merged, df_products, left_on=['product_id'], right_on=['product_id'])
      
      # storing in DW
      target.write('Gerando tabela pedidos no DW\n')
      
      df_merged.to_sql('pedidos', index=False, con=conn_dw, if_exists='replace')

      target.write('Finalizando processo ETL\n')

In [10]:
# run luigi as local scheduler
luigi.build([OrderTask()], local_scheduler=True)

DEBUG: Checking if OrderTask() is complete
DEBUG: Checking if ProductsTask() is complete
INFO: Informed scheduler that task   OrderTask__99914b932b   has status   PENDING
DEBUG: Checking if CustomerTask() is complete
INFO: Informed scheduler that task   ProductsTask__99914b932b   has status   PENDING
INFO: Informed scheduler that task   CustomerTask__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 3
INFO: [pid 57] Worker Worker(salt=948709935, workers=1, host=ea88237e6e30, username=root, pid=57) running   CustomerTask()
INFO: [pid 57] Worker Worker(salt=948709935, workers=1, host=ea88237e6e30, username=root, pid=57) done      CustomerTask()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   CustomerTask__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 57] Worker Worker(salt=948709935

True

In [11]:
sql = """
  select
    p.contact_name as cliente_name,
    sum(p.subtotal)
  from pedidos p
  group by p.contact_name
  order by subtotal desc
"""
df = pd.read_sql_query(sql, conn_dw)
df

,cliente_name,sum(p.subtotal)
0,Georg Pipps,10033.280040
1,Horst Kloss,11950.080001
2,Karl Jablonski,2938.199998
3,Carlos Hernández,3242.820006
4,Anabela Domingues,1296.000062
...,...,...
62,Fran Wilson,712.000007
63,Maurizio Moroni,80.100001
64,Francisco Chang,100.799999
65,Jytte Petersen,352.599998
